In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings("ignore")


In [3]:
data_path = "../result_data_files/corrected_length.csv"
df = pd.read_csv(data_path)

In [4]:
def replace_values(df, column_name, replace_with, min, max):
    # Замяна на стойности по-малки или равни на 0
    df[column_name] = df[column_name].apply(lambda x: replace_with if x <= min else x)
    
    # Замяна на стойности, равни на 255
    df[column_name] = df[column_name].apply(lambda x: replace_with if x >= max else x)
    
    return df

In [5]:
df = replace_values(df, 'Скорост', np.nan, 0, 255)

In [6]:
df = replace_values(df, 'Тегло', np.nan, 0, 115000)

In [7]:
df = replace_values(df, 'Дължина', np.nan, 1000, 20000)

In [8]:
def edit_data(df_for_month):
    df_for_month = replace_values(df_for_month, 'Скорост', np.nan, 0, 255)
    df_for_month = replace_values(df_for_month, 'Тегло', np.nan, 0, 115000)
    df_for_month = replace_values(df_for_month, 'Дължина', np.nan, 1000, 20000)
    return df_for_month

In [9]:
df_1 = edit_data(df)

In [10]:
def resample_half_hour(df, datetime_column, columns_to_include, agg_funcs,how_much):
    
    # Увери се, че колоната с дата е във формат datetime
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    
    # Избиране само на необходимите колони
    df = df[[datetime_column] + columns_to_include]
    
    # Задаване на колоната с дата като индекс
    df.set_index(datetime_column, inplace=True)
    
    # Групиране на данните на всеки половин час и прилагане на агрегационни функции
    df_resampled = df.resample(how_much).agg(agg_funcs)
    
    # Добавяне на нова колона с броя на записите за всеки интервал
    df_resampled['Брой автомобили'] = df.resample(how_much).size()
    
    # Възстановяване на индекса като колона с дата
    df_resampled = df_resampled.reset_index()

    return df_resampled

In [ ]:
how_much=["15min","30min","1H","2H","4H","6H","12H","24H"]
for how_much_2 in how_much:
    # Ресемплиране на данните с текущия елемент от масива how_much
    df_resampled = resample_half_hour(
        df_1, 
        'date', 
        columns_to_include=['Скорост', 'Тегло', 'Дължина'], 
        agg_funcs={'Скорост': 'mean', 'Тегло': 'mean', 'Дължина': 'mean'},how_much=how_much_2
    )
    
    # Записване в CSV файл с име, базирано на текущия елемент
    df_resampled.to_csv(
        f'../aggregated_data_files/year_with_date_{how_much_2}.csv',
        index=False, 
        date_format='%Y-%m-%d'
    )
    
    # Записване в Excel файл с име, базирано на текущия елемент
    df_resampled.to_excel(
        f'../aggregated_data_files/year_with_date_final_{how_much_2}.xlsx',
        index=False
    )
